In [5]:
from os import listdir
# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text
# specify directory to load
directory_train_neg = '/Users/hanxintian/Downloads/nlP/aclImdb/train/neg'
directory_train_pos = '/Users/hanxintian/Downloads/nlP/aclImdb/train/pos'
directory_test_pos = '/Users/hanxintian/Downloads/nlP/aclImdb/test/pos'
directory_test_neg = '/Users/hanxintian/Downloads/nlP/aclImdb/test/neg'
doc_train_pos = []
doc_train_neg = []
doc_test_pos = []
doc_test_neg = []
for filename in listdir(directory_train_neg):
    # create the full path of the file to open
    path = directory_train_neg + '/' + filename
    # load document
    doc = load_doc(path)
    doc_train_neg.append(doc)
for filename in listdir(directory_train_pos):
    # create the full path of the file to open
    path = directory_train_pos + '/' + filename
    # load document
    doc = load_doc(path)
    doc_train_pos.append(doc)
    
for filename in listdir(directory_test_neg):
    # create the full path of the file to open
    path = directory_test_neg + '/' + filename
    # load document
    doc = load_doc(path)
    doc_test_neg.append(doc)

for filename in listdir(directory_test_pos):
    # create the full path of the file to open
    path = directory_test_pos + '/' + filename
    # load document
    doc = load_doc(path)
    doc_test_pos.append(doc)

In [6]:
# use 20000 data as train and 5000 as valid
train_data = doc_train_neg[:20000].append(doc_train_pos[:20000])
valid_data = doc_test

12500